In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import warnings
from src.portfolio import FractilePortfolio, PureFactorPortfolio
from src.enums import Universe
from src.analysis import PortfolioAnalysis
import plotly.graph_objects as go
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")
from concurrent.futures import ProcessPoolExecutor, as_completed

In [3]:
def get_percentile(df):
    df = df.fillna(np.inf).drop("Date", axis=1)
    percentile_month  = df.quantile([0.05, 0.25, 0.50])/ 30.25
    return percentile_month.T

Fractile Analysis

In [3]:
fractile_df = pd.read_excel('results/FractilePortfolio.xlsx')
pure_df =  pd.read_excel('results/PureFactorPortfolio.xlsx')

In [4]:
get_percentile(fractile_df)

c:\Users\bapdu\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Users\bapdu\anaconda3\Lib\site-packages\numpy\lib\function_base.py:4658: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t >= 0.5,


,0.05,0.25,0.50
Momentum,3.014876,4.619835,5.900826
Value,3.900826,20.826446,NaN
Quality,20.102479,NaN,NaN
Low Volatility,12.294215,21.834711,29.652893
Market,12.476033,22.636364,NaN


In [17]:
get_percentile(pure_df)

,0.05,0.25,0.50
Momentum,3.504132,5.421488,6.297521
Value,1.464463,6.966942,20.000000
Quality,6.016529,12.033058,25.107438
Low Volatility,7.095868,11.223140,18.115702
Market,7.117355,10.867769,17.867769


## Strategy

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import warnings
from src.portfolio import FractilePortfolio, PureFactorPortfolio
from src.enums import Universe
from src.analysis import PortfolioAnalysis
import plotly.graph_objects as go
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")
from concurrent.futures import ProcessPoolExecutor, as_completed

In [2]:

start_date = "2019-12-31"
end_date = "2024-12-31"
ptf_analysis = PortfolioAnalysis(universe = Universe.CAC40)
sensi_factors = ["Momentum","Value","Quality","Low Volatility","Market"]
sensis = ["Momentum"]
portfolio = PureFactorPortfolio

combined_results, liste_results = [results_basic_strat, results_half_date_strat] = ptf_analysis.compare_strategies(
                        target_factor="Momentum",
                        sensi_factors=sensi_factors,
                        start_date_str=start_date,
                        end_date_str=end_date,
                        Portfolio=portfolio,
                        transaction_fees = 0.0005
                    )

# basic_strat, half_life_strat, undesired_strategy= liste_results

display(combined_results.df_statistics.head(10))

combined_results.ptf_value_plot.show()
combined_results.ptf_drawdown_plot.show() 

for rebalancing_plot in combined_results.plt_rebalancing_plot:
    rebalancing_plot.show()

,Metrics,Momentum Basic strategy PureFactorPortfolio,Momentum Half life strategy PureFactorPortfolio,Momentum Undesired strategy PureFactorPortfolio
0,Total Return,16.03%,27.32%,21.04%
1,Annualized Return,2.91%,4.77%,3.76%
2,Volatility,7.22%,7.55%,7.28%
3,Sharpe Ratio,0.40,0.63,0.52
4,Sortino Ratio,0.50,0.85,0.71
5,Max Drawdown,-12.13%,-13.30%,-12.47%
6,VaR 95%,-0.71%,-0.72%,-0.74%
7,CVaR 95%,-1.13%,-1.13%,-1.09%
8,Frais de transaction,2.22%,0.72%,0.62%


In [3]:
combined_results.df_statistics.head(10).to_excel('cac_momentum.xlsx')

In [7]:
liste_results[2].df_statistics

,Metrics,Momentum Undesired strategy PureFactorPortfolio
0,Total Return,21.41%
1,Annualized Return,3.96%
2,Volatility,5.40%
3,Sharpe Ratio,0.73
4,Sortino Ratio,0.95
5,Max Drawdown,-12.34%
6,VaR 95%,-0.53%
7,CVaR 95%,-0.82%
8,Frais de transaction,0.84%


## Sauvegarde des poids

In [8]:
weights = liste_results[2].ptf_weights

In [ ]:
# === 3. Transformation en format 'long' (Date - Ticker - Quantity) ===

NOTIONNEL = 1  # Montant notionnel par date

df_invested = weights * NOTIONNEL

df_long = df_invested.reset_index().melt(id_vars=df_invested.index.name, 
                                         var_name='Identifier', 
                                         value_name='Amount')

# === 4. Nettoyage : supprimer NaN ou poids nuls ===
df_long = df_long.dropna()
df_long = df_long[df_long['Amount'] != 0]

weights.index.name = 'Date'

# Maintenant, reset_index() fonctionnera correctement
weights_long = weights.reset_index().melt(id_vars='Date', 
                                          var_name='Identifier', 
                                          value_name='Amount')
weights_long = weights_long.dropna()
weights_long = weights_long[weights_long['Amount'] != 0]


# Réorganisation finale
weights_long = weights_long[['Date', 'Identifier', 'Amount']]

df = weights_long.sort_values('Date')
df['Month'] = df['Date'].dt.to_period('M')
# Garder la dernière ligne par mois et par ticker
df_mensuel = df.groupby(['Month', 'Identifier']).tail(1)

# Remettre une date standard pour Bloomberg (début du mois ou autre)
df_mensuel['Date'] = df_mensuel['Month'].dt.to_timestamp()

# Réorganisation
df_mensuel = df_mensuel[['Date', 'Identifier', 'Amount']]
df_mensuel.to_excel('poids_final_momentum_bloomberg.xlsx', index=False)

In [10]:
df_mensuel

,Date,Identifier,Amount
306972,2020-01-01,COO UN Equity,-0.006487
57888,2020-01-01,SYK UN Equity,-0.010719
66694,2020-01-01,WY UN Equity,0.005158
428998,2020-01-01,PAYX UW Equity,0.008947
125820,2020-01-01,MTD UN Equity,0.002847
...,...,...,...
215117,2024-12-01,MCK UN Equity,-0.001751
213859,2024-12-01,JBHT UW Equity,0.003919
77995,2024-12-01,VTR UN Equity,-0.004012
228955,2024-12-01,PPL UN Equity,0.007627
